# 5.2 Local Search y optimizacion
**Vctor misael Escalante Alvarado**

Complejidad O(I x A x(EP X N^2 x E x R))
* I: Número de instancias.
* A: Número de algoritmos (Hill Climber, ILS, SA, etc.).
* R: Número de ejecuciones por algoritmo.
* N: Número de nodos.
* E: Número de aristas.
* EP: Número máximo de evaluaciones permitidas.

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

# Aqui va la lectura del grafo
def leer_grafo(filepath):
    # Lee el archivo de texto y construye una representación del grafo como una lista de adyacencia y extrae información adicional si está presente en la primera línea.
    with open(filepath, 'r') as f:
        lines = f.readlines()

    # Ignorar encabezados y espacios vacíos
    datos_grafo = [line.strip() for line in lines if line.strip() and not line.startswith("Nombre")]

    # La primera línea útil indica el número de nodos, el mejor resultado conocido y el número de aristas
    total_nodos, mejor_resultado, total_aristas = map(int, datos_grafo[0].split())

    # Leer las conexiones desde la segunda línea en adelante
    lista_conexiones = [tuple(map(int, line.split())) for line in datos_grafo[1:]]

    # Identificar todos los nodos únicos
    conjunto_nodos = sorted(set(u for edge in lista_conexiones for u in edge))

    return conjunto_nodos, mejor_resultado, total_aristas, lista_conexiones

def save_results_to_file(instance, algorithm_name, costs):
    # Guarda los resultados en un archivo de texto.
    filename = f"{instance}_{algorithm_name}_results.txt"
    with open(filename, 'w') as file:
        file.write("\n".join(map(str, costs)))
    print(f"Resultados guardados en {filename}")

# Escribe un costo individual en el archivo de resultados
def save_partial_result_to_file(filename, cost):
    with open(filename, 'a') as file:
        file.write(f"{cost}\n")

# La función del objetivo
def objective_function(solution, conexiones, nodos):
    posiciones = {nodo: solution.index(nodo) for nodo in nodos}  # Mapeo nodo -> posición
    return max(abs(posiciones[u] - posiciones[v]) for u, v in conexiones)

# Aqui se hace el Hill Climber--------------------------------------------------------------
def visitaVecindario_FirstImp(solution, cost_function):
    size = len(solution)
    order = np.random.permutation(size)  # Orden aleatorio de nodos
    for i in order:
        for j in order:
            if i != j:
                neighbor = solution[:]
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]  # Intercambiar los nodos
                if cost_function(neighbor) < cost_function(solution):
                    return neighbor
    return solution

def visitaVecindario_BestImp(solution, cost_function):
    size = len(solution)
    order = np.random.permutation(size)
    best_neighbor = solution[:]
    best_cost = cost_function(solution)
    for i in order:
        for j in order:
            if i != j:
                neighbor = solution[:]
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]  # Intercambiar los nodos
                neighbor_cost = cost_function(neighbor)
                if neighbor_cost < best_cost:
                    best_neighbor = neighbor
                    best_cost = neighbor_cost
    return best_neighbor

def hill_climber(conexiones, nodos, steps, evaluations_limit, improvement_type):
    # Generar una permutación inicial de nodos
    solution_actual = nodos[:]
    random.shuffle(solution_actual)
    current_cost = objective_function(solution_actual, conexiones, nodos)
    evaluations = 0

    for _ in range(steps):
        if evaluations >= evaluations_limit:
            break

        if improvement_type == "firstImprovement":
            nueva_solucion = visitaVecindario_FirstImp(solution_actual, lambda s: objective_function(s, conexiones, nodos))
        elif improvement_type == "bestImprovement":
            nueva_solucion = visitaVecindario_BestImp(solution_actual, lambda s: objective_function(s, conexiones, nodos))

        nuevo_costo = objective_function(nueva_solucion, conexiones, nodos)
        evaluations += 1

        if nuevo_costo < current_cost:
            solution_actual = nueva_solucion
            current_cost = nuevo_costo
        else:
            break  # Óptimo local
    return current_cost

def ils(conexiones, nodos, evaluations_limit):
    def perturb(solution):
        i, j = random.sample(range(len(solution)), 2)
        solution[i], solution[j] = solution[j], solution[i]
        return solution

    # Inicialización
    solucion_actual = nodos[:]
    random.shuffle(solucion_actual)
    best_solution = solucion_actual[:]
    best_cost = objective_function(solucion_actual, conexiones, nodos)
    evaluations = 1

    while evaluations < evaluations_limit:
        # Perturbar y mejorar
        nueva_solucion = perturb(solucion_actual[:])
        nueva_solucion = visitaVecindario_FirstImp(nueva_solucion, lambda s: objective_function(s, conexiones, nodos))
        nuevo_costo = objective_function(nueva_solucion, conexiones, nodos)
        evaluations += 1

        # Aceptar si es mejor o no
        if nuevo_costo < best_cost:
            best_solution = nueva_solucion[:]
            best_cost = nuevo_costo
        solucion_actual = nueva_solucion[:]

    return best_cost

def sa(conexiones, nodos, evaluations_limit, t_initial=100.0, alpha=0.99):
    # Iniciarlo
    solution = nodos[:]
    random.shuffle(solution)
    best_solution = solution[:]
    best_cost = objective_function(solution, conexiones, nodos)
    current_cost = best_cost
    T = t_initial
    evaluations = 1

    while evaluations < evaluations_limit and T > 1e-4:
        # Generar vecino al azar
        neighbor = solution[:]
        i, j = random.sample(range(len(solution)), 2)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        neighbor_cost = objective_function(neighbor, conexiones, nodos)
        evaluations += 1

        # Aceptar vecino basado en la temperatura
        if neighbor_cost < current_cost or random.random() < np.exp((current_cost - neighbor_cost) / T):
            solution = neighbor[:]
            current_cost = neighbor_cost

        # Actualizar mejor solución
        if current_cost < best_cost:
            best_solution = solution[:]
            best_cost = current_cost

        # Enfriar
        T *= alpha
    return best_cost


def read_values_from_file(filename):
    #Lee los valores de un archivo de texto y los devuelve como una lista de floats.
    with open(filename, 'r') as file:
        return [float(line.strip()) for line in file.readlines()]

def calculate_rmse_individual(values, target_value):
    #Calcula el RMSE para cada valor individual con el objetivo
    return [np.sqrt((value - target_value) ** 2) for value in values]


Este codigo de preferencia solo ejecutarlo 1 vez y ya

In [ ]:
#Ejecucion y guardar los grafos

# Parámetros
instances = ["bcspwr02.txt"]
evaluations_limit = 1000
algorithms = {
    "HillClimber-FirstImprovement": lambda conexiones, nodos, limit: hill_climber(conexiones, nodos, 250, limit, "firstImprovement"),
    "HillClimber-BestImprovement": lambda conexiones, nodos, limit: hill_climber(conexiones, nodos, 250, limit, "bestImprovement"),
    "ILS": ils,
    "SA": sa
}
num_executions = 20

# Almacenar resultados
all_results = {instance: {algo: [] for algo in algorithms} for instance in instances}

for instance in instances:
    print(f"Procesando instancia: {instance}")
    nodos, target_value, num_aristas, conexiones = leer_grafo(instance)

    for algo_name, algo_func in algorithms.items():
        print(f"  Ejecutando algoritmo: {algo_name}")
        filename = f"{instance}_{algo_name}_results.txt"

        # Limpiar archivo anterior si existe
        open(filename, 'w').close()

        for _ in range(num_executions):
            cost = algo_func(conexiones, nodos, evaluations_limit)

            # Guardar en el archivo de resultados
            save_partial_result_to_file(filename, cost)

            # También guardar en memoria
            all_results[instance][algo_name].append(cost)

Procesando instancia: bcspwr02.txt
  Ejecutando algoritmo: HillClimber-FirstImprovement
  Ejecutando algoritmo: HillClimber-BestImprovement
  Ejecutando algoritmo: ILS


Aqui es para imprimir los resultados de los archivos creados

In [ ]:
#Los calculos de RMSE en de los archivos

instances = ["bcspwr02.txt"]
algorithms = ["HillClimber-FirstImprovement", "HillClimber-BestImprovement", "ILS", "SA"]

# Estructura para almacenar los RMSE individualmente
rmse_data = {instance: {algo_name: [] for algo_name in algorithms} for instance in instances}

for instance in instances:
    # Leer los objectivos de lo sarchivos
    _, target_value, _, _ = leer_grafo(instance)

    for algo_name in algorithms:
        # Leer los resultados desde el archivo correspondiente
        result_filename = f"{instance}_{algo_name}_results.txt"
        costs = read_values_from_file(result_filename)

        # Calcular RMSE individuales y almacenarlos
        rmse_values = calculate_rmse_individual(costs, target_value)
        rmse_data[instance][algo_name] = rmse_values

#Aqui se hacen los grafos
for instance in instances:
    # Preparar los datos para el gráfico
    data = []
    labels = []

    for algo_name in algorithms:
        # Obtener la lista de RMSE para cada algoritmo
        rmse_values = rmse_data[instance][algo_name]
        data.append(rmse_values)  # Matplotlib espera listas anidadas
        labels.append(algo_name)

    # Crear el boxplot para las comparaciones
    plt.figure(figsize=(12, 6))
    #plt.boxplot(data, labels)  # Reemplazar lo s"labels" por los "tick_labels" y hacerlos marcar correctamente
    #plt.boxplot(data, tick_labels=labels)  # Reemplazar lo s"labels" por los "tick_labels" y hacerlos marcar correctamente
    plt.boxplot(data, labels=["HillClimber-FirstImprovement", "HillClimber-BestImprovement", "ILS", "SA"])  # Reemplazar lo s"labels" por los "tick_labels" y hacerlos marcar correctamente
    plt.title(f"Comparación de RMSE con los algoritmo para la instancia {instance}")
    plt.ylabel("RMSE")
    plt.xlabel("Algoritmo")
    plt.grid()
    plt.show()